In [1]:
import pandas as pd

# 파일 경로 설정
file_path = r'D:\project2\산업용주요고객판매량분석_차성윤\data\output_v2.csv'

df = pd.read_csv(file_path, encoding='utf-8')
    
df.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2258233 entries, 0 to 2258232
Data columns (total 31 columns):
 #   Column     Dtype  
---  ------     -----  
 0   구분         object 
 1   용도         object 
 2   번지순번       int64  
 3   설치장소번호     int64  
 4   사용계약번호     int64  
 5   시설물번호      object 
 6   고객번호       float64
 7   고객명        object 
 8   도로명주소      object 
 9   지번주소       object 
 10  시군구        object 
 11  등급         float64
 12  최초공급일      object 
 13  업종분류       object 
 14  업종         object 
 15  상품명        object 
 16  상품계약일자     object 
 17  상품해지일자     object 
 18  월 사용예정량    object 
 19  청구년월       object 
 20  관리회차       int64  
 21  사용량(m3)    float64
 22  사용량(mj)    float64
 23  납기구분       object 
 24  검침적용일자     object 
 25  공동주택코드     float64
 26  공동주택명      object 
 27  매출년월       object 
 28  정제된 도로명주소  object 
 29  위도         float64
 30  경도         float64
dtypes: float64(7), int64(4), object(20)
memory usage: 534.1+ MB


In [2]:
df.head()

,구분,용도,번지순번,설치장소번호,사용계약번호,시설물번호,고객번호,고객명,도로명주소,지번주소,...,사용량(m3),사용량(mj),납기구분,검침적용일자,공동주택코드,공동주택명,매출년월,정제된 도로명주소,위도,경도
0,가스요금,업무용,10166830,2000478035,6000522362,UM09590191,5.000350e+09,사회복지법인대명어린이집,"남구 대명역1길 53, 대명어린이집 (대명동 1051-2)",남구 대명6동 1051-2 대명어린이집,...,28.7332,1223.1149,일반,2020-07-01,NaN,NaN,2020-07-01,대구광역시 남구 대명역1길 53,35.838834,128.567554
1,가스요금,업무용,10166830,2000478035,6000522362,UM09590191,5.000350e+09,사회복지법인대명어린이집,"남구 대명역1길 53, 대명어린이집 (대명동 1051-2)",남구 대명6동 1051-2 대명어린이집,...,29.7240,1270.2254,일반,2020-09-01,NaN,NaN,2020-09-01,대구광역시 남구 대명역1길 53,35.838834,128.567554
2,가스요금,영업용,10303068,2000112321,6002005254,UM06212373,5.000726e+09,비에이치씨치킨,"달서구 호산로29길 106, 상가 (파호동 82-34)",달서구 파호동 82-34 상가,...,210.0496,8957.1451,일반,2020-02-01,NaN,NaN,2020-02-01,대구광역시 달서구 호산로29길 106,35.852039,128.476693
3,가스요금,영업용,10303068,2000112321,6002005254,UM06212373,5.000726e+09,비에이치씨치킨,"달서구 호산로29길 106, 상가 (파호동 82-34)",달서구 파호동 82-34 상가,...,219.9576,9363.1551,일반,2020-07-01,NaN,NaN,2020-07-01,대구광역시 달서구 호산로29길 106,35.852039,128.476693
4,가스요금,영업용,10185993,2000110810,6002730777,UM06210860,5.001501e+09,땅땅치킨 동인점,"북구 침산로 153, 명성푸르지오상가 103동 B5호 (침산동 269-10)",북구 침산3동 269-10 명성푸르지오상가 103동 B5호,...,3.9632,169.3872,일반,2020-09-01,NaN,NaN,2020-09-01,대구광역시 북구 침산로 153,35.889058,128.590483


In [4]:
import pandas as pd
import os

# 1. 경로 및 파일 설정
base_path = r'D:\project2\산업용주요고객판매량분석_차성윤\data'
input_file = os.path.join(base_path, 'output_v2.csv')
output_file = os.path.join(base_path, 'industry_yearly_summary.csv')

print("🚀 데이터 요약 및 정제 작업을 시작합니다...")

# 2. 필요한 열만 지정 (메모리 절약)
# 원본 열 이름: 상품명, 고객명, 매출년월, 사용량(m3), 사용량(mj)
cols_to_use = ['상품명', '고객명', '매출년월', '사용량(m3)', '사용량(mj)']

try:
    # 데이터가 크므로 Chunk 단위로 읽어서 처리합니다.
    chunk_list = []
    chunk_size = 100000
    
    for chunk in pd.read_csv(input_file, encoding='utf-8', usecols=cols_to_use, chunksize=chunk_size):
        # (1) 산업용 데이터만 필터링
        filtered_chunk = chunk[chunk['상품명'] == '산업용'].copy()
        
        # (2) 매출년월에서 연도 추출
        filtered_chunk['매출년월'] = pd.to_datetime(filtered_chunk['매출년월'])
        filtered_chunk['매출년도'] = filtered_chunk['매출년월'].dt.year
        
        # (3) 1차 그룹화 (메모리 확보를 위해 청크별로 미리 합산)
        summary_chunk = filtered_chunk.groupby(['고객명', '매출년도']).agg({
            '사용량(m3)': 'sum',
            '사용량(mj)': 'sum'
        }).reset_index()
        
        chunk_list.append(summary_chunk)
    
    # 3. 모든 청크 합치고 최종 그룹화
    final_df = pd.concat(chunk_list)
    final_summary = final_df.groupby(['고객명', '매출년도']).agg({
        '사용량(m3)': 'sum',
        '사용량(mj)': 'sum'
    }).reset_index()
    
    # 열 이름 알기 쉽게 변경
    final_summary.columns = ['고객명', '매출년도', '사용량', '사용열량']

    # 4. 최종 결과 저장
    final_summary.to_csv(output_file, index=False, encoding='utf-8-sig')
    
    print(f"✅ 작업 완료! 요약 데이터가 저장되었습니다: {output_file}")
    print(f"📊 요약된 데이터 행 개수: {len(final_summary)}")

    # 결과 확인
    display(final_summary.head(10))

except Exception as e:
    print(f"❌ 오류 발생: {e}")

🚀 데이터 요약 및 정제 작업을 시작합니다...
✅ 작업 완료! 요약 데이터가 저장되었습니다: D:\project2\산업용주요고객판매량분석_차성윤\data\industry_yearly_summary.csv
📊 요약된 데이터 행 개수: 3864


,고객명,매출년도,사용량,사용열량
0,(복)미망인모자복지회,2020,4999.5768,2.130610e+05
1,(복)미망인모자복지회,2021,116.9144,4.980486e+03
2,(유)대안에이엔씨,2020,54642.0000,2.328163e+06
3,(유)대안에이엔씨,2021,68119.0000,2.902948e+06
4,(유)대안에이엔씨,2022,37650.0000,1.614371e+06
5,(유)대안에이엔씨,2023,37076.0000,1.584256e+06
6,(유)대안에이엔씨,2024,38776.0000,1.655115e+06
7,(유)대안에이엔씨,2025,42660.0000,1.819569e+06
8,(재)대구테크노파크 F&B,2020,78088.0000,3.326858e+06
9,(재)대구테크노파크 F&B,2021,89234.0000,3.801861e+06
